In [1]:
import pandas as pd
import plotly.plotly as py
import plotly.graph_objs as go
import numpy as np

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from tqdm import tqdm_notebook

In [2]:
init_notebook_mode(connected=True)

In [27]:
df_all = pd.read_csv('00_data/elections_round2_sim_all_v9.csv')
print(df_all.shape)
df_all.head()

(29989, 61)


,district_number,district_gen_numeric,district_general,district_location,"2.К-сть виборців, внесених до списку",ЗеленськийВолодимир,ПорошенкоПетро,participation_round1,participation_delta,participation_round2,...,ЗеленськийВолодимир_inflow_other_cands,ПорошенкоПетро_inflow_others,ПорошенкоПетро_inflow_БойкоЮрій,ПорошенкоПетро_inflow_ВілкулОлександр,ПорошенкоПетро_inflow_ГриценкоАнатолій,ПорошенкоПетро_inflow_КошулинськийРуслан,ПорошенкоПетро_inflow_ЛяшкоОлег,ПорошенкоПетро_inflow_СмешкоІгор,ПорошенкоПетро_inflow_ТимошенкоЮлія,ПорошенкоПетро_inflow_other_cands
0,50130,11,"Вінницька область, Територіальний виборчий окр...","Школа-ліцей, спортивна зала",1686,232,370,0.653025,0.108205,0.761230,...,23.133912,137.821570,8.838257,3.038151,20.162274,4.695324,12.428799,33.143464,46.677045,8.838257
1,50131,11,"Вінницька область, Територіальний виборчий окр...","ДНЗ ""Казка"", спортивна зала",2109,319,365,0.635846,0.089095,0.724941,...,28.548822,289.322978,20.697382,14.532204,42.715874,10.128506,20.257012,64.734365,93.798774,22.458861
2,50132,11,"Вінницька область, Територіальний виборчий окр...","ПСП ""Агрофірма Батьківщина"", приіщення контори",1282,204,247,0.646646,0.137611,0.784256,...,26.461357,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,50133,11,"Вінницька область, Територіальний виборчий окр...","школа, спортзал",2434,320,524,0.649959,0.105591,0.755550,...,22.397456,419.689520,31.235350,16.469548,53.384053,17.605379,34.642843,105.632274,131.188470,29.531604
4,50134,11,"Вінницька область, Територіальний виборчий окр...","звіргосподарство, кабінет головного бухгалтера",225,36,39,0.622222,0.073363,0.695585,...,5.037945,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [26]:
#credentials needed
py.sign_in('okuchma', '1X4AOENqX8gafp4qNWcS')

In [19]:
candidates = ['ЗеленськийВолодимир','ПорошенкоПетро']

base_cols = ['district_number', 'district_gen_numeric']

main_cands = ['БойкоЮрій',
             'ВілкулОлександр',
             'ГриценкоАнатолій',
             'КошулинськийРуслан',
             'ЛяшкоОлег',
             'СмешкоІгор',
             'ТимошенкоЮлія',
             'other_cands']

In [20]:
no_vote = 'не голосували'
indices = [i for i in range(len(candidates+main_cands+[no_vote]+candidates+[no_vote]))]
indices = [i for i in zip([outflow for outflow in (candidates+main_cands+[no_vote])]+
                          [inflow for inflow in (candidates+[no_vote])],indices)]

indices

[('ЗеленськийВолодимир', 0),
 ('ПорошенкоПетро', 1),
 ('БойкоЮрій', 2),
 ('ВілкулОлександр', 3),
 ('ГриценкоАнатолій', 4),
 ('КошулинськийРуслан', 5),
 ('ЛяшкоОлег', 6),
 ('СмешкоІгор', 7),
 ('ТимошенкоЮлія', 8),
 ('other_cands', 9),
 ('не голосували', 10),
 ('ЗеленськийВолодимир', 11),
 ('ПорошенкоПетро', 12),
 ('не голосували', 13)]

In [21]:
labels = [i[0] for i in indices]

outflows = []
for index in indices[:11]:
    outflows += [index for i in range(3)]
    
inflows = [] 
for index in range(len(indices[:11])):
    inflows += indices[-3:]
    
colors_dict = {'ЗеленськийВолодимир':'#3ec258',
               'ПорошенкоПетро':'#c2443e',
               'не голосували':'#666666'}

colors = [colors_dict[label] if label in colors_dict else '#ffdb83' for label in labels]

print(len(outflows),len(inflows))
source = [outflow[1] for outflow in outflows]
target = [inflow[1] for inflow in inflows]

33 33


In [22]:
for candidate in candidates:
    df_all[candidate+'_inflow_'+candidate] = df_all[candidate] - df_all[candidate+'_stayed_home'] - df_all[candidate+'_outflow']

df_all[candidates[0]+'_inflow_'+candidates[1]] = df_all[candidates[0]+'_inflow']
df_all[candidates[1]+'_inflow_'+candidates[0]] = df_all[candidates[1]+'_inflow']

df_all.columns

Index(['district_number', 'district_gen_numeric', 'district_general',
       'district_location', '2.К-сть виборців, внесених до списку',
       'ЗеленськийВолодимир', 'ПорошенкоПетро', 'participation_round1',
       'participation_delta', 'participation_round2', 'voted_round2',
       'zel_share', 'ЗеленськийВолодимир_round2', 'ПорошенкоПетро_round2',
       'БойкоЮрій', 'ВілкулОлександр', 'ГриценкоАнатолій',
       'КошулинськийРуслан', 'ЛяшкоОлег', 'СмешкоІгор', 'ТимошенкоЮлія',
       'other_cands', 'pool_round1', 'stayed_home', 'mobilized',
       'voted_round1', 'ЗеленськийВолодимир_stayed_home',
       'ПорошенкоПетро_stayed_home', 'stayed_home_others',
       'БойкоЮрій_stayed_home', 'ВілкулОлександр_stayed_home',
       'ГриценкоАнатолій_stayed_home', 'КошулинськийРуслан_stayed_home',
       'ЛяшкоОлег_stayed_home', 'СмешкоІгор_stayed_home',
       'ТимошенкоЮлія_stayed_home', 'other_cands_stayed_home',
       'ЗеленськийВолодимир_mobilized', 'ПорошенкоПетро_mobilized',
      

In [23]:
def count_values(outflow,inflow):
    outflow_name = outflow[0].replace('_outflow','')
    inflow_name = inflow[0].replace('_inflow','')
    no_vote1_outflow = df_all['2.К-сть виборців, внесених до списку'].sum()-df_all['voted_round1'].sum()-df_all['mobilized'].sum()
    
    print(f'From candidate {outflow_name} to candidate {inflow_name}')
    print(inflow_name+'_inflow_'+outflow_name)
    
    if (outflow_name==no_vote)|(inflow_name==no_vote):
        if (outflow_name==no_vote)&(inflow_name==no_vote):
            print(no_vote1_outflow)
            values.append(no_vote1_outflow)
        elif outflow_name==no_vote:
            print(df_all[inflow_name+'_mobilized'].sum())
            values.append(df_all[inflow_name+'_mobilized'].sum())
        else:
            print(df_all[outflow_name+'_stayed_home'].sum())
            values.append(df_all[outflow_name+'_stayed_home'].sum())
    else:
        print(df_all[inflow_name+'_inflow_'+outflow_name].sum())
        values.append(df_all[inflow_name+'_inflow_'+outflow_name].sum())
    
values = []
for index,value in enumerate(outflows):
    count_values(outflows[index],inflows[index])
sum(values)

From candidate ЗеленськийВолодимир to candidate ЗеленськийВолодимир
ЗеленськийВолодимир_inflow_ЗеленськийВолодимир
5614630
From candidate ЗеленськийВолодимир to candidate ПорошенкоПетро
ПорошенкоПетро_inflow_ЗеленськийВолодимир
12785
From candidate ЗеленськийВолодимир to candidate не голосували
не голосували_inflow_ЗеленськийВолодимир
86619
From candidate ПорошенкоПетро to candidate ЗеленськийВолодимир
ЗеленськийВолодимир_inflow_ПорошенкоПетро
16448
From candidate ПорошенкоПетро to candidate ПорошенкоПетро
ПорошенкоПетро_inflow_ПорошенкоПетро
2953715
From candidate ПорошенкоПетро to candidate не голосували
не голосували_inflow_ПорошенкоПетро
44446
From candidate БойкоЮрій to candidate ЗеленськийВолодимир
ЗеленськийВолодимир_inflow_БойкоЮрій
724974.5989140808
From candidate БойкоЮрій to candidate ПорошенкоПетро
ПорошенкоПетро_inflow_БойкоЮрій
1235439.9960062222
From candidate БойкоЮрій to candidate не голосували
не голосували_inflow_БойкоЮрій
245812.62669355347
From candidate ВілкулОлек

30047225.54785839

In [24]:
np_source = np.array(source)
np_values = np.array(values)
np_maxes = np.array([])

for triplet in range(np_source.max()+1):
    np_maxes = np.append(np_maxes,np_values[np.where(np_source==triplet)].max())

color_links = np.array(['rgba(204,204,204,0.3)' for i in range(len(np_source))])
color_links[np.where(np.isin(np_values,np_maxes))[0]]='rgba(204,204,204,1)'

data = {'type':'sankey',
       'node':{'label':labels,
               'color':colors,
               'pad':10,
               'line':{'width':0},
               'hovertemplate': "голосів"
              },
       'link':{'source':source,
              'target':target,
              'value':values,
              'color':color_links,
              'hovertemplate': "голосів"}
       }

fig = {'data':[data]}
py.iplot(fig, validate=False)
#iplot(fig, validate=False)

In [42]:
['ЗеленськийВолодимир', 'ПорошенкоПетро',
'ЗеленськийВолодимир_stayed_home', 'ПорошенкоПетро_stayed_home',
'ЗеленськийВолодимир_outflow', 'ПорошенкоПетро_outflow',
'ЗеленськийВолодимир_inflow', 'ПорошенкоПетро_inflow',
'ЗеленськийВолодимир_inflow_others','ПорошенкоПетро_inflow_others',
'ЗеленськийВолодимир_mobilized', 'ПорошенкоПетро_mobilized']

candidates = ['Зеленський','Порошенко']
stayed_home = [df_all['ЗеленськийВолодимир_stayed_home'].sum()*(-1),df_all['ПорошенкоПетро_stayed_home'].sum()*(-1)]
outflow = [df_all['ЗеленськийВолодимир_outflow'].sum()*(-1),df_all['ПорошенкоПетро_outflow'].sum()*(-1)]
base = [df_all['ЗеленськийВолодимир'].sum(),df_all['ПорошенкоПетро'].sum()]
mobilization = [df_all['ЗеленськийВолодимир_mobilized'].sum(),df_all['ПорошенкоПетро_mobilized'].sum()]
inflow_competitor = [df_all['ЗеленськийВолодимир_inflow'].sum(),df_all['ПорошенкоПетро_inflow'].sum()]
inflow_others = [df_all['ЗеленськийВолодимир_inflow_others'].sum(),df_all['ПорошенкоПетро_inflow_others'].sum()]

labels_bar = ['0','2 млн','4 млн','6 млн','8 млн','10 млн']
tickvals = [0,2*10**6,4*10**6,6*10**6,8*10**6,10*10**6]

trace1 = go.Bar(
    y=candidates,
    x=stayed_home,
    orientation = 'h',
    marker = dict(
        color = ['rgba(62,194,88,0)','rgba(194,68,62,0)']
    )
)

trace2 = go.Bar(
    y=candidates,
    x=outflow,
    orientation = 'h',
    marker = dict(
        color = ['rgba(62,194,88,0)','rgba(194,68,62,0)']
    )
)

trace3 = go.Bar(
    y=candidates,
    x=base,
    orientation = 'h',
    marker = dict(
        color = ['rgba(62,194,88,1)','rgba(194,68,62,1)']
    )
)

trace4 = go.Bar(
    y=candidates,
    x=mobilization,
    orientation = 'h',
    marker = dict(
        color = ['rgba(62,194,88,0.8)','rgba(194,68,62,0.8)']
    )
)

trace5 = go.Bar(
    y=candidates,
    x=inflow_competitor,
    orientation = 'h',
    marker = dict(
        color = ['rgba(62,194,88,0.6)','rgba(194,68,62,0.6)']
    )
)

trace6 = go.Bar(
    y=candidates,
    x=inflow_others,
    orientation = 'h',
    marker = dict(
        color = ['rgba(62,194,88,0.6)','rgba(194,68,62,0.6)']
    )
)

data = [trace1, trace2, trace3, trace4, trace5, trace6]
layout = go.Layout(
    barmode='stack',
    showlegend=False,
    xaxis=go.layout.XAxis(
        ticktext=labels_bar,
        tickvals=tickvals
    )
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)
#iplot(fig)